<a id='top'></a>
# JWST Spectroscopic Data Calibration: Pipeline Stage 2
---
**Authors**:  
Maria Pena-Guerrero (pena@stsci.edu), Jo Taylor (jotaylor@stsci.edu)

Based on original notebooks by Bryan Hilbert (hilbert@stsci.edu)

**Latest Update**: 29 May 2021

<div class="alert alert-block alert-warning">
    <h3><u><b>Notebook Goals</b></u></h3>
<br>
Working with the Spectroscopic Stage 2 Calibration Pipeline, we will:
<ul>
    <li>Look at the different ways to call the pipeline</li>
    <li>Examine exactly what each pipeline step does to the science data</li>
    <li>Look in detail at the steps in common to all spectroscopic modes</li>
</ul>
<br>
<b><i>IMPORTANT</b></i>: The goal of this notebook is not to actually run the pipeline, since each step is only applied to specific JWST modes, but rather to illustrate <i>what</i> each step is doing and <i>how</i> you would run it if you need to. Any code below is provided to showcase how to run the pipeline or any steps, and is not intended to actually be executed.
</div>

## Table of Contents
* [Introduction](#intro)
* [Spectroscopy Modes](#spec_modes)
* [Pipeline Resources and Documentation](#resources)
   * [Installation](#installation)
   * [Reference Files](#reference_files)
* [Imports](#imports)
* [Methods for calling steps/pipelines](#calling_methods)
* [Parameter reference files](#parameter_reffiles)
* [calwebb_spec2](#spec2) 
   * [Run the entire pipeline](#spec2_at_once)
       * [run() method](#run_method)
       * [call() method](#call_method)
       * [command line](#command_line)
   * [Run the individual pipeline steps](#spec2_step_by_step)
       * [The `Assign WCS` step](#assign_wcs)
       * [The `Background Subtraction` step](#background)
       * [The `Imprint Subtraction` step](#imprint)
       * [The `MSA Flagging` step](#msaflagging)
       * [The `2D Extraction` step](#extract_2d)
       * [The `Source Type` step](#srctype)
       * [The `Master Background` step](#masterbg)
       * [The `Wavecorr` step](#wavecorr)
       * [The `Flat Field` step](#flat_field)
       * [The `Straylight` step](#straylight)
       * [The `Fringe Correctoin` step](#fringe)
       * [The `Pathloss` step](#pathloss)
       * [The `Barshadow` step](#barshadow)
       * [The `Photom` step](#photom)
       * [The `Resample Spec` step](#resample)
       * [The `Cube Build` step](#cube_build)
       * [The `1D Extraction` step](#extract_1d)


<a id='intro'></a>
## Introduction

This notebook covers the Stage 2 processing of the JWST spectroscopic data, also known as *calwebb_spec2*. This material should serve as a reference guide when you need to run *calwebb_spec2*, or any of its constituent steps, in the future. 

**_IMPORTANT_**: The goal of this notebook is not to actually run the pipeline, since each step is only applied to specific JWST modes, but rather to illustrate *what* each step is doing and *how* you would run it if you need to. Any code below is provided to showcase how to run the pipeline or any steps, and is not intended to actually be run.

*Calwebb_spec2* is the “Swiss army knife” of JWST pipelines. It contains corrections that are only applied to certain instruments and/or instrument modes. The pipeline is a wrapper which will string together the appropriate steps for each exposure in the proper order.

The [*calwebb_spec2* pipeline](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_spec2.html#calwebb-spec2) applies corrections to slope images (countrate files) which are the output files of the [Stage 1 pipeline](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_detector1.html#calwebb-detector1). These slope images will be in the form of either *_rate* files (mean countrate across all integrations) or *_rateints* files (one countrate image per integration). A single input file can be processed or an association file listing multiple inputs can be supplied, in which case the processing steps will be applied to each input exposure one at a time. If *_rateints* products are used as input, each step applies its algorithm to each integration in the exposure, where appropriate.

The main output from *calwebb_spec2* is a fully calibrated, but unrectified, exposure with suffix *_cal*, in units of MJy/str (or MJy for NIRSpec or NIRISS SOSS point sources). The exact datamodel type of this file will depend on the spectrocpic mode in use, see the [documentation](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_spec2.html#d-or-3d-calibrated-data) for complete details.

Additionally, 1D extracted spectral products are also created. For non-TSO observations, these are *_x1d* files containing extracted spectra for one or more slits/sources. For TSO observations, these are 3D *_x1dints* files with extracted spectra for each integration.

Finally, 2D or 3D rectified products (*_s2d* and *s3d*) may also be produced depending on the mode in use. However, these files are meant only for user examination of the data. It is the *_cal.fits* files that are passed on to Stage 3 of the pipeline.

<a id='spec_modes'></a>
## Spectroscopy Modes

The subset, and order, of steps run by *calwebb_spec2* depends on the mode of the input JWST exposure. There are eight different spectroscopic instrument mode combinations, listed below:
* NIRSpec FS = Fixed Slit
* NIRSpec MOS = Multi-Object Spectroscopy
* NIRSpec IFU = Integral Field Unit
* MIRI FS = LRS (Low Resolution Spectroscopy) Fixed Slit
* MIRI SL = LRS Slitless
* MIRI MRS = Medium Resolution Spectroscopy (IFU)
* NIRISS SOSS = Single Object Slitless Spectroscopy
* NIRISS and NIRCam WFSS = Wide-Field Slitless Spectroscopy

The list of all *calwebb_spec2* steps and which modes they apply to can be found [here](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html#calwebb-spec2).

<a id='resources'></a>
## Pipeline Resources and Documentation

There are several different places to find information on installing and running the pipeline. This notebook will summarize each Stage 2 spectroscopic pipeline step. To find more in-depth instructions use the links below.

* [JWST Documentation (JDox) for the Stage 2 spectroscopic pipeline](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/stages-of-processing/calwebb_spec2) including short a short summary of what each step does.

* [High-level description of all pipeline stages and steps](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html) from the pipeline software documentation website.

* [`jwst` package documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html) including how to run the pipeline, input/output files, etc.

* [`jwst` package GitHub repository, with installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md)

* [**Help Desk**](https://stsci.service-now.com/jwst?id=sc_cat_item&sys_id=27a8af2fdbf2220033b55dd5ce9619cd&sysparm_category=e15706fc0a0a0aa7007fc21e1ab70c2f): **If you have any questions or problems regarding the pipeline, submit a ticket to the Help Desk**

<a id='installation'></a>
### Installation

<div class="alert alert-block alert-info">
    During the JWebbinar, we will be working in a pre-existing environment where the <b>jwst</b> package has already been installed, so you won't need to install it yourself.
</div>

<div class="alert alert-block alert-warning">
If you wish to run this notebook outside of this JWebbinar, you will have to first install the <b>jwst</b> package.<br>

For more detailed instructions on the various ways to install the package, see the [installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md) on GitHub.

The easiest way to install the pipeline is via `pip`. Below we show how to create a new conda environment, activate that environment, and then install the latest released version of the pipeline. You can name your environment anything you like. In the lines below, replace `<env_name>` with your chosen environment name.

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install jwst`

If you wish to install the development version of the pipeline, which is more recent than (but not as well tested compared to) the latest released version:

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install git+https://github.com/spacetelescope/jwst`
    
</div>

<a id='reference_files'></a>
### Reference Files

[Calibration reference files](https://jwst-docs.stsci.edu/data-processing-and-calibration-files/calibration-reference-files) are a collection of FITS and ASDF files that are used to remove instrumental signatures and calibrate JWST data. For example, the dark current reference file contains a multiaccum ramp of dark current signal to be subtracted from the data during the dark current subtraction step. 

When running a pipeline or pipeline step, the pipeline will automatically look for any required reference files in a pre-defined local directory. If the required reference files are not present, they will automatically be downloaded from the Calibration Reference Data System (CRDS) at STScI.

<div class="alert alert-block alert-info">
    During the JWebbinar, our pre-existing existing environment is set up to correctly use and store calibration reference files, and you do not need to set the environment variables below.
</div>
    
<div class="alert alert-block alert-warning">
If you wish to run this notebook outside of this JWebbinar, you will have to specify a local directory in which to store reference files, along with the server to use to download the reference files from CRDS. To accomplish this, there are two environment variables that should be set prior to calling the pipeline. These are the CRDS_PATH and CRDS_SERVER_URL variables. In the example below, reference files will be downloaded to the "crds_cache" directory under the home directory.

>`$ export CRDS_PATH=$HOME/crds_cache`<br>
>`$ export CRDS_SERVER_URL=https://jwst-crds.stsci.edu`<br>
OR:<br>
`os.environ["CRDS_PATH"] = "/user/myself/crds_cache"`<br>
`os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"`<br>

The first time you run the pipeline, the CRDS server should download all of the context and reference files that are needed for that pipeline run, and dump them into the CRDS_PATH directory. Subsequent executions of the pipeline will first look to see if it has what it needs in CRDS_PATH and anything it doesn't have will be downloaded from the STScI cache. 
</div>

<div class="alert alert-block alert-warning">NOTE: Users at STScI should automatically have access to the Calibration Reference Data System (CRDS) cache for running the pipeline, and can skip setting these environment variables.</div>

[Top of Notebook](#top)

<a id='imports'></a>
## Imports

Import packages to illustrate what is needed to run the pipeline and each of its steps.

In [ ]:
# Packages that allow us to get information about objects:
import asdf
import os

# Astropy tools:
from astropy.io import fits

# JWST data models
from jwst import datamodels

# The entire calwebb_spec2 pipeline
from jwst.pipeline.calwebb_spec2 import Spec2Pipeline

# Individual steps that make up calwebb_spec2 and datamodels
from jwst.assign_wcs.assign_wcs_step import AssignWcsStep
from jwst.background.background_step import BackgroundStep
from jwst.imprint.imprint_step import ImprintStep
from jwst.msaflagopen.msaflagopen_step import MSAFlagOpenStep
from jwst.extract_2d.extract_2d_step import Extract2dStep
from jwst.srctype.srctype_step import SourceTypeStep
from jwst.master_background.master_background_step import MasterBackgroundStep
from jwst.wavecorr.wavecorr_step import WavecorrStep
from jwst.flatfield.flat_field_step import FlatFieldStep
from jwst.straylight.straylight_step import StraylightStep
from jwst.fringe.fringe_step import FringeStep
from jwst.pathloss.pathloss_step import PathLossStep
from jwst.barshadow.barshadow_step import BarShadowStep
from jwst.photom.photom_step import PhotomStep
from jwst.resample import ResampleSpecStep
from jwst.cube_build.cube_build_step import CubeBuildStep
from jwst.extract_1d.extract_1d_step import Extract1dStep

Check which version of the pipeline we are running:

In [ ]:
import jwst
print(jwst.__version__)

Define the output directory.

In [ ]:
# To make everything easier, all files saved by the pipeline
# and pipeline steps will be saved to the working directory.
output_dir = './'

# Make sure the output directory exists before downloading any data
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

[Top of Notebook](#top)

<a id='calling_methods'></a>
## Methods for calling steps/pipelines

There are three common methods by which the pipeline or pipeline steps can be called. From within python, the `run()` and `call()` methods of the pipeline or step classes can be used. Alternatively, the `strun` command can be used from the command line. Examples of each method are shown below.

When using the `call()` method or `strun`, optional input parameters can be specified via [parameter reference files](#parameter_reffiles). When using the `run()` method, these parameters are instead specified within python.

<a id='parameter_reffiles'></a>
## Parameter Reference Files

When calling a pipeline or pipeline step using the `call()` method or the command line, [parameter reference files](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/config_asdf.html#config-asdf-files) can be used to specify values for input parameters. These reference files are in [asdf](https://asdf.readthedocs.io/en/stable/) format and appear somewhat similar to json files when examined in a text editor. 

Versions of parameter reference files containing default parameter values for each step and pipeline are available in CRDS. When using the `call()` method, if you do not specify a parameter reference file name in the call, the pipeline or step will retrieve and use the appropriate file from CRDS, which will then run the pipeline or step with the parameter values in that file. If you provide the name of a parameter reference file, then the parameter values in that file will take precedence. For any parameter not specified in your parameter reference file, the pipeline will use the default value.

When using `strun`, the parameter reference file is a required input.

Let's take a look at the contents of a parameter reference file. We'll open it using the asdf package, and use the `tree` attribute to see what's inside:

In [ ]:
step = Spec2Pipeline()
spec2_param_reffile = os.path.join(output_dir, 'calwebb_spec2.asdf')
step.export_config(spec2_param_reffile)
spec2_reffile = asdf.open(spec2_param_reffile)

In [ ]:
spec2_reffile.info(max_rows=None)
# or
#spec2_reffile.tree

The top part of the file contains various metadata entries about the file itself. Below that, you'll see a `'name'` entry, which lists `Spec2Pipeline` as the class to which these parameters apply. The next line contains the `parameters` entry, which lists parameters and values attached to the pipeline itself. Below this is the `steps` entry, which contains a list of dictionaries. Each dictionary refers to one step within the pipeline, and specifies parameters and values that apply to that step. If you look through these entries, you'll see the same parameters and values that we specified manually when using the `run()` method above.

In [ ]:
# Don't forget to close the file
spec2_reffile.close()

[Top of Notebook](#top)

---
<a id='spec2'></a>
## The calwebb_spec2 pipeline: spectroscopic processing

In the sections below, we will illustrate how to run the Stage 2 pipeline on a single input rate file. We will first show how to call the entire [*calwebb_spec2* pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html#calwebb-spec2) on the file. The pipeline is a wrapper which wil']l string together all of the appropriate steps in the proper order. 

The final output from this call is a flux-calibrated exposure image which is ready to go into the Stage 3 pipeline.

See [Figure 1](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/stages-of-processing/calwebb_spec2) on the calwebb_spec2 algorithm page for a map of which steps are performed depending on spectroscopic mode.

<a id='spec2_at_once'></a>
### Run the entire `calwebb_spec2` pipeline

In this section we show how to run the entire calwebb_spec2 pipeline with a single call using the `run()`, `call()`, and `strun` methods. In this case the pipeline code can determine which instrument was used to collect the data and runs the appropriate steps in the proper order.

You may set parameter values for some of the individual steps, save some outputs, etc, and then call the pipeline.

<a id='run_method'></a>
#### Call the pipeline using the run() method

When using the `run()` method to execute a pipeline (or step), the pipeline class is first instantiated without the data to be processed. Optional input parameters are specified using attributes of the class instance. Finally, the call to the `run()` method is made and the data are supplied.  See here for [more examples of the run() method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_run.html).

The `run()` method does not take any kind of parameter reference file as input. If you wish to set values for various parameters, you must do that manually. Below, we set several parameters in order to show how it's done. 

How do you know what parameters are available to be set and what their default values are? The `spec` property for individual steps will list them. The property is less useful for the pipelines themselves, as it does not show the parameters for the steps comprising the pipeline.

All steps and pipelines have several common parameters that can be set. 

* `save_results` specifies whether or not to save the output of that step/pipeline to a file. The default is False.
* `output_dir` is the directory into which the output files will be saved.
* `output_file` is the base filename to use for the saved result. Note that each step/pipeline will add a custom suffix onto output_file. 

<a id='spec2_using_run'></a>

<div class="alert alert-block alert-info">
Here's how you would run the entire <i>calwebb_spec2</i> pipeline. The output can be a little overwhelming. There will be multiple log entries printed to the screen for each step.
</div>

In [ ]:
# Create an instance of the pipeline class
spec2 = Spec2Pipeline()

# Set some parameters that pertain to the entire pipeline
spec2.save_results = True
spec2.output_dir = output_dir

# Here you could some parameters that pertain to some of the individual steps
spec2.extract_2d.mmag_extract = 20 # Minimum magnitude of objects to extract for WFSS data
spec2.extract_1d.bkg_fit = 'poly' # Fit a polynomial to the background values for each column or row

# Call the run() method
result = spec2.run(rate_file)

<a id='call_method'></a>
#### Call the pipeline using the call() method

When using the `call()` method, a single command will instantiate and run the pipeline (or step). The input data and optional parameter reference files are supplied in this single command. In this case, any desired input parameters cannot be set after instantiation, as with the `run()` method. See here for [example usage of call() method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_call.html).

The commands below will call the pipeline using the `call()` method and will supply the parameter reference file. 

<div class="alert alert-block alert-info">

<b>Method #1:</b>
<br>
Provide the name of the observation file, the pipeline-specific input paramters, and the name of the parameter reference file that specifies step-specific parameters
</div>

In [ ]:
call_output = Spec2Pipeline.call(rate_file,
                                 output_dir=output_dir,
                                 save_results=True,
                                 config_file=spec2_param_reffile)

<div class="alert alert-block alert-info">

<b>Method #2:</b>
<br>
In this case, build a nested dictionary that specifies parameter values for various steps, and provide it in the call to call().
</div>

In [ ]:
parameter_dict = {"extract_2d": {"mmag_extract": 20},
                  "extract_1d": {"bkg_fit": "poly"},
                 }
call_output = Spec2Pipeline.call(rate_file, output_dir=output_dir, save_results=True, steps=parameter_dict)

<a id='command_line'></a>
#### Call the pipeline using the command line

Calling a pipeline or step from the command line is similar to using the `call()` method. The data file to be processed, along with an optional parameter reference file and optional parameter/value pairs can be provided to the `strun` command. See here for [additional examples of command line calls](https://jwst-pipeline.readthedocs.io/en/stable/jwst/introduction.html?highlight=%22command%20line%22#running-from-the-command-line).

The cells below contains two different command line commands that use `strun` to call the calwebb_spec2 pipeline. 

<div class="alert alert-block alert-info">

<b>Method #1:</b>
<br>
We provide the name of the pipeline class, the observation file, and explicitly set pipeline- and step-specific parameters. You can see that the command quickly becomes quite large with the added parameter settings. 
    
```
    strun jwst.pipeline.Spec2Pipeline myinput_rate.fits --save_results=True --output_dir='./' --steps.extract_2d.mmag_extract=20 --steps.extract_1d.bkg_fit='poly'
```
</div>

<div class="alert alert-block alert-info">

<b>Method #2:</b>
<br>
This version of the command is much more succinct, as the parameter values to be set are all contained within the parameter reference file. The pipeline class is also contained in the parameter reference file, so there is no need to specify it in the command itself.
    
```
    strun spec2_modified_paramfile.asdf myinput_rate.fits
```
</div>

[Top of Notebook](#top)

<a id='detector1_step_by_step'></a>
## Run the individual pipeline steps

In the sections below we explain what each step does and illustrate how to run it. We show only the `run()` method of executing steps, but the `call()` or `strun` methods could be used with the same syntax as shown above.

<a id='assign_wcs'></a>
### The `Assign WCS` step

#### Summary

This step transforms positions in the detector frame to positions in a world coordinate system (WCS) - ICRS and wavelength. There may be intermediate coordinate frames depending on the instrument (e.g. from detector to slit frame). The WCS is saved in the ASDF extension of the FITS file, and it can be accessed as an attribute of the meta object when the fits file is opened as a datamodel. More information can be found at [assign_wcs](https://jwst-pipeline.readthedocs.io/en/stable/jwst/assign_wcs/index.html).

#### Affected Modes
This step is run for the following modes:
- NIRSpec: all
- MIRI: all
- NIRISS: all
- NIRCam: all

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/assign_wcs/main.html) of the step.

#### Arguments

There are no optional arguments for this step.

#### Reference files used

This step uses various reference files, see [`assign_wcs` reference files](https://jwst-pipeline.readthedocs.io/en/stable/jwst/assign_wcs/reference_files.html). 


#### Run the step

In [ ]:
# Using the run() method. Instantiate and set parameters
step = AssignWcsStep()
step.save_results = True
step.output_dir = output_dir

# Call the run() method
result = step.run(rate_file)

[Top of Notebook](#top)

<a id='background'></a>
## The `Background` step

#### Summary

The background subtraction step performs image-from-image subtraction to remove the background signal. The step takes as input one target exposure and one or more background exposures, which are then subtracted from the target exposure. The method of background image subtraction varies depending on observing mode. 

For most spectroscopic modes, the background exposure (or averaged background exposure, if more than one is provided) is subtracted from the SCI array of the target exposure and the DQ arrays of the background and target exposures are combined using a bitwise OR operation. For WFSS exposures, a background reference image is scaled and subtracted from the target exposure.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/background/description.html) of the step.

#### Affected Modes
This step is run for the following modes:
- NIRSpec: all
- MIRI: FS, MRS
- NIRISS: all
- NIRCam: all

#### Arguments

There are two optional arguments for this step that are only used for non-WFSS exposures. They are used in the process of creating an average background image, to control the sigma clipping. Please see the documentation for further details: [optional arguments](https://jwst-pipeline.readthedocs.io/en/latest/jwst/background_step/arguments.html).

#### Reference files used

This step uses reference files (`WFSSBKG` and `WAVELENGTHRANGE`) only when processing WFSS exposures, see [WFSS background reference files](https://jwst-pipeline.readthedocs.io/en/stable/jwst/background/reference_files.html).

#### Run the step

In [ ]:
# Using the run() method. Instantiate and set parameters
step = BackgroundStep()
step.save_results = True
step.output_dir = output_dir
bkg_list = [background_img1, background_img2]

# Call the run() method
result = step.run(previous_result, bkg_list)

 <a id='imprint'> </a>
## The `Imprint Subtraction` step

#### Summary

This is a NIRSpec-specific step, applied only to the MSA and IFU modes. It removes patterns created by the MSA's grid of metal shutters. Pattern removal is done by subtracting a dedicated exposure taken with all MSA shutters closed and the IFU entrance aperture blocked. This step requires two input parameters, the target exposure and the imprint exposure.

#### Affected Modes
This step is run for the following modes:
- NIRSpec: MOS, IFU
- MIRI: none
- NIRISS: none
- NIRCam: none

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/imprint/description.html) of the step.

#### Arguments

There are no optional arguments for this step.

#### Reference files used

This step uses no specific reference files.


#### Run the step

In [ ]:
# Using the run() method. Instantiate and set parameters
step = ImprintStep()
step.save_results = True
step.output_dir = output_dir
msa_imprint = 'myinput_msa_imprint.fits'

# Call the run() method
result = step.run(previous_result, msa_imprint)

[Top of Notebook](#top)

<a id='msaflagging'></a>
## The `MSA Flagging` step

#### Summary

This is a NIRSpec-specific step, applied only to the MSA and IFU modes. It flags pixels that are affected by MSA shutters that are stuck in the open position. 

#### Affected Modes
This step is run for the following modes:
- NIRSpec: MOS, IFU
- MIRI: none
- NIRISS: none
- NIRCam: none

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/msaflagopen/description.html) of the step.

#### Arguments

There are no optional arguments for this step.

#### Reference files used

This step uses the [`MSAOPER` reference file](https://jwst-pipeline.readthedocs.io/en/stable/jwst/msaflagopen/reference_files.html).

#### Run the step

In [ ]:
# Instantiate and set parameters
step = MSAFlagOpenStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(previous_result)

[Top of Notebook](#top)

<a id='extract_2d'></a>
## The `2D Extraction` step 

#### Summary

This step extracts 2D arrays from spectral images. The extractions are performed on all of the SCI, ERR, and DQ arrays of the input dataset, as well as any variance arrays that may be present. 

It also computes an array of wavelengths to attach to the extracted data. The extracted arrays are stored as one or more slit objects in an output `MultiSlitModel` and saved as separate tuples of extensions in the output FITS file.

#### Affected Modes
This step is run for the following modes:
- NIRSpec: FS, MOS
- MIRI: none
- NIRISS: WFSS
- NIRCam: all

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/extract_2d/main.html) of the step.

#### Arguments

This step has several [optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/extract_2d/main.html#step-arguments).

#### Reference files used
For NIRISS and NIRCam WFSS modes, this step uses the [`WAVELENGTHRANGE` reference file](https://jwst-pipeline.readthedocs.io/en/stable/jwst/extract_2d/reference_files.html).


#### Run the step

In [ ]:
# Instantiate and set parameters
step = Extract2dStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(previous_result)

[Top of Notebook](#top)

<a id='srctype'></a>
## The `Source Type` step 

#### Summary

This step estimates if a spectroscopic source should be considered to be a point or extended object, and populates the “SRCTYPE” main header keyword with a value of either “POINT” or “EXTENDED.” This information is then used in subsequent spectroscopic processing steps to apply source-dependent corrections.

#### Affected Modes
This step is run for the following modes:
- NIRSpec: all
- MIRI: all
- NIRISS: SOSS
- NIRCam: none

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/srctype/description.html)

#### Arguments

There are no optional arguments for this step.

#### Reference files used

This step uses no specific reference files.


#### Run the step

In [ ]:
# Instantiate and set parameters
step = SourceTypeStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(result)

<a id='masterbg'></a>
##  The `Master Background Subtraction` step 

#### Summary

This step is typically run during Stage 3 spectroscopic processing (*calwebb_spec3*) for all modes **except** NIRSpec MOS, for which the step is instead run in *calwebb_spec2*.

In general, the master background subtraction method works by taking a 1D background spectrum, interpolating it back into the 2D space of a science image, and then subtracting it. This allows for higher SNR background data to be used than what might be obtainable by doing direct image-from-image subtraction using only one or a few background images. The 1D master background spectrum can either be constructed on-the-fly by the calibration pipeline using available background data or supplied by the user.

#### Affected Modes
This step is run for the following modes:
- NIRSpec: MOS
- MIRI: none
- NIRISS: none
- NIRCam: none

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/master_background/description.html) of the step.

#### Arguments

This steps has multiple optional arguments- for further details please see the [documentation](https://jwst-pipeline.readthedocs.io/en/stable/jwst/master_background/arguments.html).


#### Reference files used

This step uses no specific reference files.


#### Run the step

In [ ]:
# Instantiate and set parameters
step = MasterBackgroundStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(result)

[Top of Notebook](#top)

<a id='wavecorr'></a>
## The `Wavecorr` step

#### Summary

This step is only applied to NIRSpec FS and MOS modes. It updates the wavelength assignments for point sources that are known to be off center (in the dispersion direction) in their slit.

For NIRSpec MOS exposures, wavelength assignments created during extract_2d step are based on a source that’s perfectly centered in a slitlet. Most sources, however, are not centered in every slitlet in a real observation. The wavecorr step loops over all slit instances in the input science product and applies a wavelength correction to slits that contain a point source.

For NIRSpec FS exposures, fixed slit data do not have an a priori estimate of the source location within a given slit, so the estimated source location is computed by the wavecorr step. Any offsets to this estimation are then computed using the same method as for MOS slitlets.

#### Affected Modes
This step is run for the following modes:
- NIRSpec: FS, MOS
- MIRI: none
- NIRISS: none
- NIRCam: none

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/wavecorr/description.html) of the step.

#### Arguments

This step has no step-specific arguments.

#### Reference files used

This step uses the [`WAVECORR` reference file](https://jwst-pipeline.readthedocs.io/en/stable/jwst/wavecorr/reference_files.html).


#### Run the step

In [ ]:
# Instantiate and set parameters
step = WavecorrStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(result)

[Top of Notebook](#top)

<a id='flat_field'></a>
## The `Flat Field` step

#### Summary

This step flat-fields an input science dataset by dividing by a flat-field reference image. The SCI array from the flat-field reference file is divided into both the SCI and ERR arrays of the science dataset, and the flat-field DQ array is combined with the science DQ array using a bitwise OR operation. 

Flat field correction for NIRSpec modes calculates an on-the-fly flat rather than using a set reference file. 

#### Affected Modes
This step is run for the following modes:
- NIRSpec: all
- MIRI: all
- NIRISS: all
- NIRCam: all

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/main.html) of the step.

#### Arguments

This step has only one optional argument, used for NIRSpec modes: [save_interpolated_flat](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/arguments.html).

#### Reference files used

This step uses several reference files: [non-NIRSpec flats](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/reference_files.html) and [NIRSpec flats](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/reference_files.html#reference-files-for-nirspec-spectroscopy).


#### Run the step

In [ ]:
# Instantiate and set parameters
step = FlatFieldStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(result)

[Top of Notebook](#top)

<a id='straylight'></a>
## The `Straylight` step

#### Summary

This step is only applied to MIRI MRS channel 1 and 2 data. The straylight correction uses information in the `REGIONS` reference file to determine which pixels belong to a slice and which pixels are located in the slice gaps. It removes stray light that may contaminate spectra by interpolating the measured signal in the inter-slice regions of the detector. The inter-slice regions nominally should not receive light from the sky and therefore should serve as a good measure of the amount of stray light within the exposure.


#### Affected Modes
This step is run for the following modes:
- NIRSpec: none
- MIRI: MRS
- NIRISS: none
- NIRCam: none

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/straylight/main.html) of the step.

#### Arguments

This step has three [optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/straylight/arguments.html), depending on which algorithm is used to correct for stray light.

#### Reference files used

This step uses the [`REGIONS` reference file](https://jwst-pipeline.readthedocs.io/en/stable/jwst/straylight/reference_files.html).


#### Run the step

In [ ]:
# Instantiate and set parameters
step = StraylightStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(result)

[Top of Notebook](#top)

<a id='fringe'></a>
## The `Fringe Correction` step

#### Summary
This step applies a fringe correction to MIRI MRS images. The SCI array from a fringe reference file is divided into the SCI and ERR arrays of the science dataset. Only pixels that have valid (non-NaN) values in the SCI array of the reference file will be corrected. The DQ and variance arrays of the science exposure are not currently modified by this step.

#### Affected Modes
This step is run for the following modes:
- NIRSpec: none
- MIRI: MRS
- NIRISS: none
- NIRCam: none

#### Documentation
[Full description](https://jwst-pipeline.readthedocs.io/en/latest/jwst/fringe/main.html) of the step.

#### Arguments
This step has no step-specific arguments.

#### Reference files used
This step uses the [`FRINGE` reference file](https://jwst-pipeline.readthedocs.io/en/latest/jwst/fringe/reference_files.html).

#### Run the step

In [ ]:
# Instantiate and set parameters
step = FringeStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(result)

[Top of Notebook](#top)

<a id='pathloss'></a>
## The `Pathloss` step

#### Summary
The pathloss step calculates and applies corrections that are needed to account for various types of signal loss in spectroscopic data. For NIRSpec, this correction accounts for losses in the optical system due to light being scattered outside the grating, and to light not passing through the aperture. For NIRISS SOSS data it corrects for the flux that falls outside the subarray. There is currently no correction for MIRI and NIRCam.

#### Affected Modes
This step is run for the following modes:
- NIRSpec: all
- MIRI: none
- NIRISS: SOSS
- NIRCam: none

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pathloss/description.html) of the step.

#### Arguments
This step has no step-specific arguments.

#### Reference files used
This step uses the [`PATHLOSS` reference file](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pathloss/reference_files.html).

#### Run the step

In [ ]:
# Instantiate and set parameters
step = PathLossStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(result)

[Top of Notebook](#top)

<a id='barshadow'></a>
## The `Barshadow` step

#### Summary

This step is only applied to NIRSpec MOS data, specifically to extended sources only. It calculates the correction due to the bar that separates adjacent microshutters. The correction values are divided into the SCI and ERR arrays, and the square of the correction values are divided into the variance arrays.

#### Affected Modes
This step is run for the following modes:
- NIRSpec: MOS
- MIRI: none
- NIRISS: none
- NIRCam: none

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/barshadow/description.html) of the step.

#### Arguments

This step has no specific arguments.

#### Reference files used

This step uses the [`BARSHADOW` reference file](https://jwst-pipeline.readthedocs.io/en/stable/jwst/barshadow/reference_files.html)


#### Run the step

In [ ]:
# Instantiate and set parameters
step = BarShadowStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(result)

[Top of Notebook](#top)

<a id='photom'></a>
## The `Photom` step

#### Summary
The photom step applied flux (photometric) calibration to convert data from units of countrate to surface brightness (MJy/Sr) or flux density (MJy). For NIRSpec and NIRISS SOSS observations of point sources, the output unit will be flux density, while all other observations will yield units of surface brightness. Pixels with wavelengths outside the range specified in each mode's reference file are set to zero and flagged as "DO_NOT_USE" in the DQ array. 

#### Affected Modes
This step is run for the following modes:
- NIRSpec: all
- MIRI: all
- NIRISS: all
- NIRCam: all

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/latest/jwst/photom/main.html) of the step.

#### Arguments
This step has no specific arguments.


#### Reference files used
The photom step uses the [`PHOTOM` and `AREA` reference files](https://jwst-pipeline.readthedocs.io/en/latest/jwst/photom/reference_files.html).



#### Run the step

In [ ]:
# Instantiate and set parameters
step = PhotomStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(result)

[Top of Notebook](#top)

<a id='resample'></a>
## The `Resample Spec` step

#### Summary

This step resamples each input 2D image based on the WCS and distortion information, and combines multiple resampled images into a single undistorted product. 

#### Affected Modes
This step is run for the following modes:
- NIRSpec: FS, MOS
- MIRI: FS
- NIRISS: none
- NIRCam: none

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html) of the step.

#### Arguments
There is a list of [optional Astrodrizzle-style](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/arguments.html) input parameters that can be used to customize the resampling process.

#### Reference files used
This step uses the [`DRIZPARS`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/reference_files.html) reference file. This file contains Astrodrizzle-style keywords that can be used to control the details of the resampling.

#### Run the step

In [ ]:
# Instantiate and set parameters
step = ResampleSpecStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(result)

[Top of Notebook](#top)

<a id='cube_build'></a>
## The `Cube Build` step 

#### Summary

This step is applied to MIRI or NIRSpec IFU calibrated 2D images to produce 3D spectral cubes. The 2D disjointed IFU slice spectra are corrected for distortion and assembled into a rectangular cube with three orthogonal axes: two spatial and one spectral.

This step is run in both the *calwebb_spec2* and *calwebb_spec3* pipelines. In *calwebb_spec2*, where the input is a single MIRI or NIRSpec IFU exposure, the default output cube will be built from all the data in that single exposure. For MIRI this means using the data from both channels (e.g. 1A and 2A) that are recorded in a single exposure and the output IFU cube will have a non-linear wavelength dimension. For NIRSpec the data is from the single grating and filter combination contained in the exposure and will have a linear wavelength dimension. The calwebb_spec2 pipeline calls cube_build with `output_type=multi`.

#### Affected Modes
This step is run for the following modes:
- NIRSpec: IFU
- MIRI: MRS
- NIRISS: none
- NIRCam: none

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/cube_build/main.html) of the step.

#### Arguments

There are a number of arguments that control the sampling size of the cube, as well as the type of data that is combined to create the cube. See all the [optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/cube_build/arguments.html).

#### Reference files used

This step uses the [`CUBEPAR` reference file](https://jwst-pipeline.readthedocs.io/en/stable/jwst/cube_build/reference_files.html).


#### Run the step

In [ ]:
# Instantiate and set parameters
step = CubeBuildStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(result)

[Top of Notebook](#top)

<a id='extract_1d'></a>
## The `1D Extraction` step 

#### Summary

This step extracts a 1D signal from a 2D or 3D dataset and writes spectral data to an output 1D extracted spectral data product. For all non-TSO modes, this is a file with suffix *_x1d*, and for TSO modes the suffix is *_x1dints*. 

For non-WFSS modes, a reference file is used to specify the location and size of the target and background extraction regions. For WFSS modes, the extraction regions are defined as the 2D subarray/cutout for each source.

#### Affected Modes
This step is run for the following modes:
- NIRSpec: all
- MIRI: all
- NIRISS: all
- NIRCam: all

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/extract_1d/description.html) of the step.

#### Arguments

This step has multiple [optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/extract_1d/arguments.html).

#### Reference files used

This step uses the [`EXTRACT1D` and `APCORR` reference files](https://jwst-pipeline.readthedocs.io/en/stable/jwst/extract_1d/reference_files.html).


#### Run the step

In [ ]:
# Instantiate and set parameters
step = Extract1dStep()
step.output_dir = output_dir
step.save_results = True

# Call the run() method
result = step.run(result)

[Top of Notebook](#top)